In [ ]:
!pip install pandas-ta xgboost scikit-learn plotly

# WEEKLY


In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go

# Step 1: Download and Resample Data
ticker = '^NSEI'  # Nifty 50 index
data = yf.download(ticker, start='2010-01-01', end='2020-01-01')
data.columns=data.columns.droplevel(1)

# Resample to weekly data
weekly_data = data.resample('W').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})
weekly_data.dropna(inplace=True)

# Step 2: Add Technical Indicators
weekly_data['sma_20'] = ta.sma(weekly_data['Close'], length=20)  # 50-week SMA
weekly_data['sma_50'] = ta.sma(weekly_data['Close'], length=50)  # 200-week SMA
weekly_data['rsi_20'] = ta.rsi(weekly_data['Close'], length=20)  # 50-week RSI
weekly_data['macd'] = ta.macd(weekly_data['Close'], fast=26, slow=52)['MACD_26_52_9']  # MACD
weekly_data['adx_50'] = ta.adx(weekly_data['High'], weekly_data['Low'], weekly_data['Close'], length=50)['ADX_50']

# Step 3: Create Target Variable (Buy, Sell, Neutral)
weekly_data['Future_Close'] = weekly_data['Close'].shift(-1)  # Predict 1 week ahead
weekly_data['Target'] = weekly_data['Future_Close'] - weekly_data['Close']
weekly_data['Signal'] = weekly_data['Target'].apply(lambda x: 'Buy' if x > 0 else 'Sell')

# Drop NaN values
weekly_data.dropna(inplace=True)

# Step 4: Features and Labels
X = weekly_data[['sma_20','sma_50','rsi_20', 'macd', 'adx_50']]
y = LabelEncoder().fit_transform(weekly_data['Signal'])  # Encode: Buy = 0 , Sell = 1

# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 6: Train XGBoost Classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Step 7: Evaluate the Model
y_pred = model.predict(X_test)

import plotly.graph_objects as go

# Step 1: Create a DataFrame for the predicted signals
y_predicted = model.predict(X_test)

# Mapping back the predictions to 'Buy'/'Sell' labels
predicted_signals = ['Buy' if pred == 0 else 'Sell' for pred in y_predicted]

# Add predicted signals to the test data
test_data = X_test.copy()
test_data['Actual_Signal'] = ['Buy' if y == 0 else 'Sell' for y in y_test]
test_data['Predicted_Signal'] = predicted_signals

# Add the actual close prices from the original data
test_data['Close'] = weekly_data.loc[test_data.index, 'Close']

# Step 2: Sort the data by Date in ascending order
test_data_sorted = test_data.sort_index(ascending=True)

# Step 3: Plot using Plotly
fig = go.Figure()

# Plot the actual close price
fig.add_trace(go.Scatter(
    x=test_data_sorted.index,
    y=test_data_sorted['Close'],
    mode='lines',
    name='Close Price',
    line=dict(color='blue', width=2)
))

# Highlight Buy signals
buy_signals = test_data_sorted[test_data_sorted['Predicted_Signal'] == 'Buy']
fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals['Close'],
    mode='markers',
    name='Buy Signal',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Highlight Sell signals
sell_signals = test_data_sorted[test_data_sorted['Predicted_Signal'] == 'Sell']
fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals['Close'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

# Update layout
fig.update_layout(
    title=f'{ticker} Stock Price with Predicted Buy and Sell Signals',
    xaxis_title='Date',
    yaxis_title='Close Price',
    template='plotly_dark',
    showlegend=True,
    height=600,
)

# Show the plot
fig.show()


[*********************100%***********************]  1 of 1 completed
